In [ ]:
import numpy as np
from dataclasses import dataclass
from scipy.optimize import least_squares, fmin
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import Dict, List
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs())
# create all the boring lists and stuff
params_list: List = []
error_list: List = []

In [ ]:
# If you want to start from the default tir file
# tire_model = readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))
# or from a existing set of parameters
tire_model = tire_model_from_arr(H_R20_18X6_7)

In [ ]:
def filer_df(runs, press, fz, tmp=50):
    df = filter_ia(filter_phit(filter_press(runs, press=press, d_press=10000)))
    df = df[np.abs(df["FZ"] - fz) < 100]
    df = df[df.TSTC > tmp]
    # df = df[df.PHIT == 0]
    df = remove_time_gaps(df)
    return df

def filt_compare(df):
    df = remove_time_gaps(filter_ia(df))
    df = df[np.abs(df.P - 82700) < 4000]
    # df = df[(np.abs(df.SL) != 0) | (np.abs(df.SA) > np.deg2rad(5))]
    # df = df[np.abs(df.SL) == 0]
    # df = df[df.TSTC < 65]
    # df = df.sample(frac=0.2)
    # df = df[df.FZ > 400]
    return df
max_sa = 15 # deg
max_pos_sr = 0.25
max_neg_sr = -0.25
vel = 11.1 # m/s
press = 82700 #68900 # Pa
fz = 1100 # N

In [ ]:
df = filt_compare(combi_runs)
combi_runs_2, cornering_2, drive_brake_2, name_2 = load_runs(get_LC0_18x6_7_runs())
df_2 = filt_compare(combi_runs_2)
df = df[np.abs(df.SA) > np.deg2rad(6)]
force_magnitude = np.sqrt(df.FX**2 + df.FY**2)
mu_magnitude = force_magnitude / df.FZ
force_magnitude_2 = np.sqrt(df_2.FX**2 + df_2.FY**2)
mu_magnitude_2 = force_magnitude_2 / df_2.FZ
# out_a = parameter_estimation_function(tire_model.dump_params()[MX_MASK_IND_PRESS], df, tire_model, params_list, error_list, fit_f_y, MX_MASK_IND_PRESS)
# out_b = parameter_estimation_function(tire_model.dump_params()[MX_MASK_IND_PRESS], df, tire_model, params_list, error_list, fit_f_x, MX_MASK_IND_PRESS)
# err_mag = np.sqrt(out_a**2 + out_b**2)
# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=df.SL, y=np.rad2deg(df.SA), z=mu_magnitude, mode='markers', marker=dict(size=1.5, color=df.TSTC, colorscale='Viridis', showscale=True), name='Data'))
# fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Error (N)', yaxis_title='Probability Density')
# fig.show()
fig = go.Figure()
# Plot FZ vs temp
fig.add_trace(go.Scattergl(x=df.TSTC, y=mu_magnitude, mode='markers', marker=dict(size=1.5), name='Mu Combi R20')) # , color=df.FZ, colorscale='Viridis', showscale=True
# fig.add_trace(go.Scattergl(x=df_2.TSTC, y=mu_magnitude_2, mode='markers', marker=dict(size=1.5), name='Mu Combi LC0'))
fig.add_trace(go.Scattergl(x=df.TSTC[df.FX>0], y=np.abs(df.FX[df.FX>0]/df.FZ[df.FX>0]), mode='markers', marker=dict(size=1.5), name='Mu Pos Fx R20'))
fig.add_trace(go.Scattergl(x=df.TSTC[df.FX<0], y=np.abs(df.FX[df.FX<0]/df.FZ[df.FX<0]), mode='markers', marker=dict(size=1.5), name='Mu Neg Fx R20'))
# fig.add_trace(go.Scattergl(x=df_2.TSTC[df_2.FX>0], y=np.abs(df_2.FX[df_2.FX>0]/df_2.FZ[df_2.FX>0]), mode='markers', marker=dict(size=1.5), name='Mu Pos Fx LC0'))
# fig.add_trace(go.Scattergl(x=df_2.TSTC[df_2.FX<0], y=np.abs(df_2.FX[df_2.FX<0]/df_2.FZ[df_2.FX<0]), mode='markers', marker=dict(size=1.5), name='Mu Neg Fx LC0'))
fig.add_trace(go.Scattergl(x=df.TSTC, y=np.abs(df.FY/df.FZ), mode='markers', marker=dict(size=1.5), name='Mu Fy R20'))
# fig.add_trace(go.Scattergl(x=df_2.TSTC, y=np.abs(df_2.FY/df_2.FZ), mode='markers', marker=dict(size=1.5), name='Mu Fy LC0'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Temp (C)', yaxis_title='Mu Magnitude', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.show()
fig = go.Figure()
# Plot temp histogram
fig.add_trace(go.Histogram(x=np.concatenate((df.TSTC, df.TSTI, df.TSTO)), histnorm='probability', name='R20'))
fig.add_trace(go.Histogram(x=np.concatenate((df_2.TSTC, df_2.TSTI, df_2.TSTO)), histnorm='probability', name='LC0'))
# fig.add_trace(go.Histogram(x=df.TSTI, histnorm='probability', name='R20'))
# fig.add_trace(go.Histogram(x=df_2.TSTI, histnorm='probability', name='LC0'))
# fig.add_trace(go.Histogram(x=df.TSTO, histnorm='probability', name='R20'))
# fig.add_trace(go.Histogram(x=df_2.TSTO, histnorm='probability', name='LC0'))
fig.add_trace(go.Histogram(x=df.RST, histnorm='probability', name='R20'))
fig.add_trace(go.Histogram(x=df_2.RST, histnorm='probability', name='LC0'))
fig.add_trace(go.Histogram(x=df.AMBTMP, histnorm='probability', name='R20'))
fig.add_trace(go.Histogram(x=df_2.AMBTMP, histnorm='probability', name='LC0'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Temp (C)', yaxis_title='Probability Density', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.show()

In [ ]:
fig = go.Figure()
df = filt_compare(combi_runs)#filer_df(combi_runs, press, fz, tmp=0) # combi_runs.copy() #
df = df[np.abs(df.SA) > 0.001]
df = df[df.FZ > 900]
dfz_real = (df.FZ - tire_model.FNOMIN) / tire_model.FNOMIN
mu_y_real = (tire_model.PDY1 * dfz_real + tire_model.PDY2)
dy_real = df.FZ * mu_y_real
ky_real = df.FY/df.SA#(np.tan(np.arcsin(df.FY / dy_real) / tire_model.PCY1) / df.SA) * tire_model.PCY1 * dy_real
fig.add_trace(go.Scattergl(x=df.TSTC, y=ky_real, mode='markers', marker=dict(size=1.5), name='Ky Combi R20'))
fig.show()
fig = go.Figure()
fig.add_trace(go.Scattergl(x=np.rad2deg(df.SA), y=ky_real, mode='markers', marker=dict(size=1.5, color=df.FZ, showscale=True), name='SA Combi R20'))
fig.show()
fig = go.Figure()
fzs = np.linspace(50, 2000, 1000)
ky = tire_model.PKY1 * tire_model.FNOMIN * np.sin(tire_model.PKY4 * np.arctan(fzs / tire_model.PKY2))
fig.add_trace(go.Scattergl(x=df.FZ, y=ky_real, mode='markers', marker=dict(size=1.5, color=np.rad2deg(df.SA), showscale=True), name='SA Combi R20'))
fig.add_trace(go.Scattergl(x=fzs, y=ky, mode='lines', name='Ky Combi R20'))
fig.show()


In [ ]:
fig = go.Figure()
fzs = np.linspace(0, 5000, 1000)
dfz = (fzs - tire_model.FNOMIN) / tire_model.FNOMIN
kx = (tire_model.PKX1 + tire_model.PKX2 * dfz) * np.exp(tire_model.PKX3 * dfz) * fzs
PKX1, PKX2, PKX3 = 48, 15, -0.3
kx_two = (PKX1 + PKX2 * dfz) * np.exp(PKX3 * dfz) * fzs
# Plot FZ vs temp
df = filt_compare(drive_brake)
df = df[np.abs(df.SA) < 0.1]
dfz_real = (df.FZ - tire_model.FNOMIN) / tire_model.FNOMIN
mu_real = (tire_model.PKX1 + tire_model.PKX2 * dfz_real)
dx_real = mu_real * df.FZ
k_real = (np.tan(np.arcsin(df.FX/dx_real)/tire_model.PCX1) / df.SL) * tire_model.PCX1 * dx_real
k = df.FX / df.SL
fig.add_trace(go.Scattergl(x=df.FZ, y=k, mode='markers', marker=dict(size=1.5, color=df.SL, showscale=True), name='K fake'))
fig.add_trace(go.Scattergl(x=df.FZ, y=k_real, mode='markers', marker=dict(size=1.5, color=df.SL, showscale=True), name='K Real'))
fig.add_scatter(x=fzs, y=kx, mode='lines', name='1.0')
fig.add_scatter(x=fzs, y=kx_two, mode='lines', name='f 1.0')
# fig.add_scatter(x=fzs, y=kx*0.5, mode='lines', name='0.5')
# fig.add_scatter(x=fzs, y=kx*0.25, mode='lines', name='0.25')
# fig.add_scatter(x=fzs, y=kx*0.15, mode='lines', name='0.15')
fig.update_yaxes(range=[-5000, 60000])
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='FZ (N)', yaxis_title='Kx (N/m)', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.show()

In [ ]:
# Make black hole graph
# a 3d surface plot of the absolute force vector magnitude as a function of slip angle and slip ratio
fig = go.Figure()
df = filer_df(combi_runs, press=press, fz=fz)
size = 100
alphas = np.linspace(-np.deg2rad(max_sa), np.deg2rad(max_sa), size)
kappas = np.linspace(max_neg_sr, max_pos_sr, size)
alphas, kappas = np.meshgrid(alphas, kappas)
alphas = alphas.flatten()
kappas = kappas.flatten()
arr_len = len(alphas)
wheel_speed = df["N"].mean()
phit = 0.0#14
inputs = np.array([np.ones(arr_len) * fz, kappas, alphas, np.zeros(arr_len), np.ones(arr_len) * phit, np.ones(arr_len) * vel, np.ones(arr_len) * press, np.ones(arr_len) * wheel_speed]).T
out = tire_model.fullSteadyState(inputs, use_turnslip=True)
fx = out[:, 0]
fy = out[:, 1]
mz = out[:, 5]
# recreate the matrix
fx = fx.reshape((size, size))
fy = fy.reshape((size, size))
mz = mz.reshape((size, size))
alphas = alphas.reshape((size, size))
kappas = kappas.reshape((size, size))
f_abs = np.sqrt(fx**2 + fy**2)
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=f_abs, colorscale='Viridis', opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=np.sqrt(df["FX"]**2 + df["FY"]**2), mode='markers', marker=dict(size=1.5, color=df.FZ, colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()
fig = go.Figure()
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fx, colorscale='Viridis', opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=df["FX"], mode='markers', marker=dict(size=1.5, color=df.FZ, colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()
fig = go.Figure()
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fy, colorscale='Viridis', opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=df["FY"], mode='markers', marker=dict(size=1.5, color=df.FZ, colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()
fig = go.Figure()
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=mz, colorscale='Viridis', opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=df["MZ"], mode='markers', marker=dict(size=1.5, color=df.FZ, colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()

In [ ]:
fig = go.Figure()
fy_tmp = df.FY.copy()
fy_tmp[df.SL != 0] = fy_tmp[df.SL != 0]# * 1.1
df_cornering = filer_df(cornering, press, fz)
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fy, opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points , colorscale='Viridis'
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=fy_tmp, mode='markers', marker=dict(size=1.5, color=np.abs(df.FZ-fz), colorscale='Viridis', showscale=True)))
# fig.add_trace(go.Scatter3d(x=np.rad2deg(df_cornering["SA"]), y=df_cornering["SL"], z=df_cornering["FY"], mode='lines')) # marker=dict(size=1.5, color=df_cornering.PHIT, colorscale='Viridis', showscale=True)
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()

In [ ]:
fig = go.Figure()
fx_tmp = df.FX.copy()
fx_tmp[df.SL != 0] = fx_tmp[df.SL != 0] #* fz * -1
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fx, opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points , colorscale='Viridis'
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=fx_tmp, mode='markers', marker=dict(size=1.5, color=np.abs(df.FZ-fz), colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()

In [ ]:
print(tire_model.RCX1)
print(tire_model.REX1)
print(tire_model.REX2)
print(drive_brake.SL)

In [ ]:
fig = go.Figure()
df = filer_df(combi_runs, press, fz, tmp=0)
fy_tmp = df.FY.copy()
fy_tmp[df.SL != 0] = fy_tmp[df.SL != 0]# * 1.1
# fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fy, opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points , colorscale='Viridis'
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=fy_tmp, mode='markers', marker=dict(size=1.5, color=df.TSTC, colorscale='Viridis', showscale=True, colorbar=dict(title="TSTC (C)"))))
fig.update_layout(template="plotly_dark", title=f"FY - {press} Pa - Fz: {fz} N - IA: 0.0 deg", margin=dict(l=0, r=0, b=0, t=50), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'), height=1000)
fig.show()

In [ ]:
fig = go.Figure()
fx_tmp = df.FX.copy()
fx_tmp[df.SL != 0] = fx_tmp[df.SL != 0]# * 1.1
# fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fx, opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points , colorscale='Viridis'
fig.add_trace(go.Scatter3d(x=np.rad2deg(df["SA"]), y=df["SL"], z=fx_tmp, mode='markers', marker=dict(size=1.5, color=df.TSTC, colorscale='Viridis', showscale=True)))
df_cornering = filer_df(drive_brake, press, fz)
# fig.add_trace(go.Scatter3d(x=np.rad2deg(df_cornering["SA"]), y=df_cornering["SL"], z=df_cornering["FX"], mode='lines'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()
print(df.TSTC.mean())

In [ ]:
print(df.keys())

In [ ]:
# plot RST and TSTC for drive and brake runs
fig = go.Figure()
df = filer_df(drive_brake, press, fz)
df = df[np.abs(df.SA + np.deg2rad(6)) < 0.01]
fig.add_trace(go.Scatter(x=df.ET, y=df.TSTC, mode='markers', name='TSTC'))
fig.add_trace(go.Scatter(x=df.ET, y=df.SA, mode='markers', name='SA'))
fig.add_trace(go.Scatter(x=df.ET, y=df.IA, mode='markers', name='IA'))
fig.add_trace(go.Scatter(x=df.ET, y=df.SL, mode='markers', name='SL'))
fig.add_trace(go.Scatter(x=df.ET, y=df.FY, mode='markers', name='FY'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Time (s)', yaxis_title='Temperature (C)')
fig.show()

In [ ]:
# make a histogram of the temperature
fig = go.Figure()
fig.add_trace(go.Histogram(x=df.TSTC, nbinsx=20, name="TSTC"))
fig.add_trace(go.Histogram(x=df.TSTI, nbinsx=20, name="TSTI"))
fig.add_trace(go.Histogram(x=df.TSTO, nbinsx=20, name="TSTO"))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), barmode='overlay')
fig.show()

In [ ]:
df2 = combi_runs.copy()#[combi_runs.FZ < 800]
# randonly sample 10%
# df2 = df2[df2.TSTC > 45]
df2 = df2[np.abs(df2.PHIT) < 0.02]
# df2 = df2[np.abs(df2.SLT) < 0.2]
# df2 = df2[df2.TSTC < 90]
df2 = df2.sample(frac=0.2)
fx_error, fy_error, mz_error = get_model_error(tire_model, df2, ts=False, abs=False)
fx_ts_error, fy_ts_error, mz_ts_error = get_model_error(tire_model, df2, ts=True)
fig = go.Figure()
# plot error as color against slip angle and slip ratio
col = np.minimum(fy_error, np.ones(df2.shape[0]) * 500)# / df.FZ
col_ts = np.minimum(fy_ts_error, np.ones(df2.shape[0]) * 500)# / df.FZ
fig.add_trace(go.Scattergl(x=df2.SA, y=df2.PHIT, mode='markers', marker=dict(size=1.5, color=col, colorscale='Viridis', showscale=True)))
fig.add_trace(go.Scattergl(x=df2.SA, y=df2.PHIT, mode='markers', marker=dict(size=1.5, color=col_ts, colorscale='Viridis', showscale=True)))
# and hoverdata for the error
fig.update_traces(marker=dict(line=dict(width=0)), hovertemplate='Fx Error: %{marker.color:.2f} N<extra></extra>')
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Pressure', yaxis_title='Vel')
# fig.show()
# create a histogram of the error
fig = go.Figure()
fig.add_trace(go.Histogram(x=fx_error, histnorm='probability density', name='Fx'))
fig.add_trace(go.Histogram(x=fy_error, histnorm='probability density', name='Fy'))
fig.add_trace(go.Histogram(x=mz_error, histnorm='probability density', name='Mz'))
fig.add_trace(go.Histogram(x=fx_ts_error, histnorm='probability density', name='Fx TS'))
fig.add_trace(go.Histogram(x=fy_ts_error, histnorm='probability density', name='Fy TS'))
fig.add_trace(go.Histogram(x=mz_ts_error, histnorm='probability density', name='Mz TS'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Error (N)', yaxis_title='Probability Density')
# fig.show()
# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=np.rad2deg(df2.SA), y=df2.SL, z=fx_error, mode='markers', marker=dict(size=1.5, color=df2.FZ, colorscale='Viridis', showscale=True)))
# fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
# fig.show()
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=np.rad2deg(df2.SA), y=df2.SL, z=fy_error, mode='markers', marker=dict(size=1.5, color=df2.TSTC, colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()

In [ ]:
split_run_fit(remove_time_gaps(combi_runs[-20000:]), tire_model, f"After Fy Optimization") # 500:10000
print(combi_runs.PHIT.max())
print(combi_runs.PHIT.min())
print(combi_runs.TSTC.max())
print(cornering.N.max())

In [ ]:
fig = go.Figure()
df = combi_runs.copy()
df = df[np.abs(df.FZ - 1100) < 100]
df = df[np.abs(df.P - press) < 3000]
df = df[np.abs(df.IA) < np.deg2rad(0.1)]
df = df[np.abs(df.SL) < 0.01]
fy_tmp = df.FY.copy()
# fy_tmp[df.SL != 0] = fy_tmp[df.SL != 0]# * 1.1
# fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=fy, opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points , colorscale='Viridis'
# fig.add_trace(go.Scatter(x=np.rad2deg(df["SA"]), y=fy_tmp, mode='lines', line=dict(color='orange', width=1)))
fig.add_trace(go.Scatter(x=np.rad2deg(df["SA"]), y=fy_tmp, mode='markers', marker=dict(size=3, color=df.TSTC, colorscale='Viridis', showscale=True, line=dict(width=0)), hovertemplate='Temp: %{marker.color:.2f} C'))
fig.add_trace(go.Scatter(x=np.rad2deg(df["SA"]), y=fy_tmp, mode='markers', marker=dict(size=3, color=np.abs(df.PHIT), colorscale='Viridis', showscale=True, line=dict(width=0)), hovertemplate='PHIT: %{marker.color:.2f} deg/m'))
fig.update_layout(barmode='overlay', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Force (N)'))
fig.show()

In [ ]:
fig = go.Figure()
max_sa = 15 # deg
max_pos_sr = 0.25
max_neg_sr = -0.25
vel = 11.1 # m/s
press = 82700 #68900 # Pa
fz = 1100 # N
df = combi_runs.copy()
df = df[10000:]
# df = df[df.TSTC > 55]
df = df[np.abs(df.FZ - fz) < 100]
df = df[np.abs(df.IA) < np.deg2rad(0.1)]
df = df[np.abs(df.P - press) < 3000]
df = df[np.abs(df.V - vel) < 2]
size = 100
alphas = np.linspace(-np.deg2rad(max_sa), np.deg2rad(max_sa), size)
kappas = np.linspace(max_neg_sr, max_pos_sr, size)
alphas, kappas = np.meshgrid(alphas, kappas)
alphas = alphas.flatten()
kappas = kappas.flatten()
arr_len = len(alphas)
wheel_speed = df["N"].mean()
phit = 0.0#14
inputs = np.array([np.ones(arr_len) * fz, kappas, alphas, np.zeros(arr_len), np.ones(arr_len) * phit, np.ones(arr_len) * vel, np.ones(arr_len) * press, np.ones(arr_len) * wheel_speed]).T
out = tire_model.fullSteadyState(inputs, use_turnslip=True)
fx = out[:, 0]
fy = out[:, 1]
mz = out[:, 5]
# recreate the matrix
fx = fx.reshape((size, size))
fy = fy.reshape((size, size))
mz = mz.reshape((size, size))
alphas = alphas.reshape((size, size))
kappas = kappas.reshape((size, size))
f_abs = np.sqrt(fx**2 + fy**2)
mu_abs = f_abs / fz
fig.add_trace(go.Surface(x=np.rad2deg(alphas), y=kappas, z=mu_abs, colorscale='Viridis', opacity=0.5, showscale=False, contours=dict(x=dict(show=True, start=-max_sa, end=max_sa, size=1), y=dict(show=True, start=max_neg_sr, end=max_pos_sr, size=0.02))))
# add the data points
fig.add_trace(go.Scatter3d(x=np.rad2deg(df.SA), y=df.SL, z=np.sqrt(df.FX**2 + df.FY**2)/df.FZ, mode='markers', marker=dict(size=1.5, color=df.FZ, colorscale='Viridis', showscale=True)))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), scene=dict(xaxis_title='Slip Angle (deg)', yaxis_title='Slip Ratio', zaxis_title='Mu abs'))
fig.show()